In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pwd

/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/notebooks/Causal Model/BEAM Parser


In [3]:
import os
import sys
# cwd = os.getcwd()
cm_folder = "/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/notebooks/Causal Model/"
models_folder = os.path.join(cm_folder, "BEAM Parser/models/")
src_path = os.path.join(cm_folder, "src")
sys.path.append(src_path)

In [4]:
from typing import Any

import dill
from sklearn.linear_model import LogisticRegression
import numpy as np

from CrossValidation import cross_validation
from MIRA import CostSensitiveMIRA
from Settings import Settings
from crel_helper import get_cr_tags
from crel_processing import essay_to_crels_cv
from evaluation import evaluate_model_essay_level, get_micro_metrics, metrics_to_df
from feature_extraction import get_features_from_probabilities
from feature_normalization import min_max_normalize_feats
from function_helpers import get_function_names
from results_procesor import ResultsProcessor
from train_parser import essay_to_crels, create_extractor_functions
from cost_functions import micro_f1_cost_plusepsilon
from train_reranker import train_model_parallel, train_model, train_cost_sensitive_instance
from window_based_tagger_config import get_config
from feature_extraction import get_features_essay_level

In [5]:
# Data Set Partition
CV_FOLDS = 5
MIN_FEAT_FREQ = 5

# Global settings
settings = Settings()
root_folder = settings.data_directory + "CoralBleaching/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
test_folder = root_folder + "Test" + "/"

coref_root = root_folder + "CoReference/"
coref_output_folder = coref_root + "CRel/"

config = get_config(training_folder)

train_fname = coref_output_folder + "training_crel_anatagged_essays_most_recent_code.dill"
with open(train_fname, "rb") as f:
    pred_tagged_essays_train = dill.load(f)

test_fname = coref_output_folder + "test_crel_anatagged_essays_most_recent_code.dill"
with open(test_fname, "rb") as f:
    pred_tagged_essays_test = dill.load(f)

print(len(pred_tagged_essays_train), len(pred_tagged_essays_test))

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/
902 226


In [6]:
cr_tags = get_cr_tags(train_tagged_essays=pred_tagged_essays_train, tag_essays_test=pred_tagged_essays_test)

set_cr_tags = set(cr_tags)
list(set_cr_tags)[0:10]

['Causer:4->Result:50',
 'Causer:1->Result:11',
 'Causer:7->Result:50',
 'Causer:50->Result:50',
 'Causer:7->Result:6',
 'Causer:2->Result:7',
 'Causer:1->Result:4',
 'Causer:3->Result:50',
 'Causer:4->Result:7',
 'Causer:5->Result:50']

In [7]:
from searn_essay_parser_breadth_first import SearnModelEssayParserBreadthFirst

In [8]:
test_folds = [(pred_tagged_essays_train, pred_tagged_essays_test)]  # type: List[Tuple[Any,Any]]
cv_folds = cross_validation(pred_tagged_essays_train, CV_FOLDS)  # type: List[Tuple[Any,Any]]

In [9]:
len(pred_tagged_essays_train)

902

### Get the Expected Crels Per Essay

In [10]:
all_essays = pred_tagged_essays_train + pred_tagged_essays_test
name2essay = {}
for essay in all_essays:
    name2essay[essay.name] = essay

name2crels = essay_to_crels(all_essays, set_cr_tags)
assert len(name2crels) == len(all_essays)

## Train Re-Ranker

In [11]:
BEAM_SIZE = 100

In [12]:
import dill

rr_fname = "xs_rerank_" + str(BEAM_SIZE) + ".dill"
with open(os.path.join(models_folder, rr_fname), "rb") as f:
    xs_rr = dill.load(f)
len(xs_rr)

902

In [13]:
assert len(xs_rr) == len(pred_tagged_essays_train)

In [14]:
len(xs_rr), len(pred_tagged_essays_train)

(902, 902)

In [15]:
%%time
xs = get_features_essay_level(xs_rr, name2crels, min_feat_freq=1)

cv_folds_rerank = cross_validation(xs, 5)
cv_folds_mm = [min_max_normalize_feats(train, test) for (train, test) in cv_folds_rerank]

CPU times: user 25.9 s, sys: 246 ms, total: 26.1 s
Wall time: 26.1 s


In [16]:
best_C, best_max_upd = 0.01, 1

In [17]:
%%time
f1 = train_model_parallel(cv_folds=cv_folds_mm, name2essay=name2essay, C=best_C, pa_type=1, loss_type="ml", \
                          max_update_items=best_max_upd, set_cr_tags=set_cr_tags)
print(f1) # 0.7372937177905368

0.733426894314066
CPU times: user 25.6 s, sys: 2.11 s, total: 27.7 s
Wall time: 2min 50s


In [18]:
%%time
f1 = train_model_parallel(cv_folds=cv_folds_mm, name2essay=name2essay, C=best_C, pa_type=1, loss_type="ml", \
                          max_update_items=best_max_upd, set_cr_tags=set_cr_tags)
print(f1) # 0.7372937177905368

0.7336580926726814
CPU times: user 25.8 s, sys: 2.13 s, total: 28 s
Wall time: 2min 55s


In [19]:
%%time
f1 = train_model_parallel(cv_folds=cv_folds_mm, name2essay=name2essay, C=best_C, pa_type=1, loss_type="ml", \
                          max_update_items=best_max_upd, set_cr_tags=set_cr_tags)
print(f1) # 0.7372937177905368

0.7338333044191401
CPU times: user 25.8 s, sys: 2.15 s, total: 27.9 s
Wall time: 2min 57s


In [20]:
BEAM_SIZE

100

## Train on Test Data

In [21]:
rr_fname = "xs_rerank_test_" + str(BEAM_SIZE) + ".dill"
with open(os.path.join(models_folder, rr_fname), "rb+") as f:
    xs_test_rerank = dill.load(f)
len(xs_test_rerank)

226

In [22]:
xs_test = get_features_essay_level(xs_test_rerank, name2crels, min_feat_freq=1)

In [23]:
# pi = xs_test[10]
# pi.all_feats_array

In [24]:
# training data comes from the test fold predictions from CV on the full training dataset
xs_train = []
for train, test in cv_folds_rerank:
    xs_train.extend(test)

In [25]:
# Normalize both using training data
xs_train_mm, xs_test_mm = min_max_normalize_feats(xs_train,xs_test)

In [26]:
num_train = int(0.8 * len(xs_train_mm))
tmp_train_copy = list(xs_train_mm)
np.random.shuffle(tmp_train_copy)
tmp_train, tmp_test = tmp_train_copy[:num_train], tmp_train_copy[num_train:]

In [27]:
%%time
C = best_C
pa_type = 1
loss_type= "ml"
max_update_items = best_max_upd

mdl = CostSensitiveMIRA(C=C, pa_type=pa_type, loss_type=loss_type, 
                        max_update_items=max_update_items, initial_weight=0.01)
# Determine number of training iterations
best_mdl, test_acc_df_ml, best_iterations = train_model(mdl, xs_train=tmp_train, xs_test=tmp_test, name2essay=name2essay, set_cr_tags=set_cr_tags,
     max_epochs=20, early_stop_iters=5, train_instance_fn = train_cost_sensitive_instance, verbose=True)

Epoch: 0 Train Accuracy: 0.7438 Test Accuracy: 0.7255
Epoch: 1 Train Accuracy: 0.7455 Test Accuracy: 0.7224
Epoch: 2 Train Accuracy: 0.7458 Test Accuracy: 0.7236
Epoch: 3 Train Accuracy: 0.7467 Test Accuracy: 0.7243
Epoch: 4 Train Accuracy: 0.7472 Test Accuracy: 0.7250
Epoch: 5 Train Accuracy: 0.7472 Test Accuracy: 0.7255
Best Test Acc: 0.7255
CPU times: user 36.4 s, sys: 132 ms, total: 36.6 s
Wall time: 36.6 s


In [28]:
best_iterations

1

In [29]:
mdl = CostSensitiveMIRA(C=C, pa_type=pa_type, loss_type=loss_type, 
                        max_update_items=max_update_items, initial_weight=0.01)

best_mdl, test_acc_df_ml,_ = train_model(mdl,  xs_train=xs_train_mm, xs_test=xs_test_mm,
                                       name2essay=name2essay, set_cr_tags=set_cr_tags,
     max_epochs=best_iterations, early_stop_iters=best_iterations, train_instance_fn = train_cost_sensitive_instance, verbose=True)

Epoch: 0 Train Accuracy: 0.7413 Test Accuracy: 0.7311
Best Test Acc: 0.7311


In [30]:
from filter_features import filter_feats

prefixes = [
    "Prob-",
    "CREL_Pair-",
    "Inv-",
    "num_crels",
    "Tally-",
    "CChain-",
    "CChainStats-",
    "Above-",
    "CREL_",
    "Propn_",
    "Diff_"
]
# xs_fltr_train, xs_fltr_test = filter_feats(xs_train_mm, xs_test_mm, prefixes)
assert len(prefixes) == len(set(prefixes)), "Duplicate prefixes found"

In [ ]:
best_f1 = -1
current_best = []
remaining = list(prefixes)

while True:
    if len(remaining) == 0:
        break
    
    f1_by_prefix = dict()
    for prefix in remaining:
        new_prefixes = current_best + [prefix]
        
        cv_filtered = []
        for tr, test in cv_folds_mm:
            x_tr,x_test = filter_feats(tr, test, new_prefixes)
            cv_filtered.append((x_tr,x_test))
        
        f1_by_prefix[prefix] = train_model_parallel(cv_folds=cv_filtered, name2essay=name2essay, C=best_C, 
                                  pa_type=1, loss_type="ml", max_update_items=best_max_upd, 
                                  set_cr_tags=set_cr_tags)
    
    best_prefix, new_best_f1 = sorted(f1_by_prefix.items(), key = lambda tpl: -tpl[1])[0]
    if new_best_f1 > best_f1:
        best_f1 = new_best_f1
        current_best.append(best_prefix)
        remaining.remove(best_prefix)
        print("{length} feats, new Best F1: {f1:.4f} Prefixes: {prefixes}".format(
            length=len(current_best), f1=best_f1, prefixes=str(current_best)))
    else:
        print("No further improvement, stopping")
        break

1 feats, new Best F1: 0.7356 Prefixes: ['Above-']
2 feats, new Best F1: 0.7388 Prefixes: ['Above-', 'num_crels']
Process stopped by user


In [ ]:
%%time
cv_filtered = []
for tr, test in cv_folds_mm:
#     x_tr,x_test = filter_feats(tr, test, prefixes)
    x_tr,x_test = filter_feats(tr, test, current_best)
    cv_filtered.append((x_tr,x_test))

f1 = train_model_parallel(cv_folds=cv_filtered, name2essay=name2essay, C=best_C, 
                          pa_type=1, loss_type="ml", max_update_items=best_max_upd, 
                          set_cr_tags=set_cr_tags)
print(f1)

## Apply to Test Data

In [ ]:
current_best

In [ ]:
xs_train_mm_fltr, xs_test_mm_fltr = filter_feats(xs_train_mm, xs_test_mm, current_best) 

In [ ]:
# Use training data to determine the best number of traininng iterations
num_train = int(0.8 * len(xs_train_mm_fltr))
tmp_train_copy = list(xs_train_mm_fltr)
np.random.shuffle(tmp_train_copy)
tmp_train, tmp_test = tmp_train_copy[:num_train], tmp_train_copy[num_train:]

In [ ]:
%%time
C = best_C
pa_type = 1
loss_type= "ml"
max_update_items = best_max_upd

mdl = CostSensitiveMIRA(C=C, pa_type=pa_type, loss_type=loss_type, 
                        max_update_items=max_update_items, initial_weight=0.01)
# Determine number of training iterations
best_mdl, test_acc_df_ml, best_iterations = train_model(mdl, xs_train=tmp_train, xs_test=tmp_test, name2essay=name2essay, set_cr_tags=set_cr_tags,
     max_epochs=20, early_stop_iters=3, train_instance_fn = train_cost_sensitive_instance, verbose=True)

In [ ]:
best_iterations 

In [ ]:
mdl = CostSensitiveMIRA(C=C, pa_type=pa_type, loss_type=loss_type, 
                        max_update_items=max_update_items, initial_weight=0.01)

best_mdl, test_acc_df_ml,_ = train_model(mdl,  
    xs_train=xs_train_mm_fltr, xs_test=xs_test_mm_fltr,
    name2essay=name2essay, set_cr_tags=set_cr_tags,
    max_epochs=best_iterations, early_stop_iters=best_iterations, 
    train_instance_fn = train_cost_sensitive_instance, verbose=True)

In [ ]:
# Set higher max iterations
mdl = CostSensitiveMIRA(C=C, pa_type=pa_type, loss_type=loss_type, 
                        max_update_items=max_update_items, initial_weight=0.01)

best_mdl, test_acc_df_ml,_ = train_model(mdl,  
    xs_train=xs_train_mm_fltr, xs_test=xs_test_mm_fltr,
    name2essay=name2essay, set_cr_tags=set_cr_tags,
    max_epochs=30, early_stop_iters=30, 
    train_instance_fn = train_cost_sensitive_instance, verbose=True)

In [ ]:
# Try no filters
mdl = CostSensitiveMIRA(C=C, pa_type=pa_type, loss_type=loss_type, 
                        max_update_items=max_update_items, initial_weight=0.01)

best_mdl, test_acc_df_ml,_ = train_model(mdl,  
    xs_train=xs_train_mm, xs_test=xs_test_mm,
    name2essay=name2essay, set_cr_tags=set_cr_tags,
    max_epochs=30, early_stop_iters=30, 
    train_instance_fn = train_cost_sensitive_instance, verbose=True)

In [ ]:
sorted(best_mdl.weights.items(), key = lambda tpl: -tpl[1])[0:20]

# Notes on Remaining Code Changes
- The Beam search approach outputs a list of Dict[str, List[float]], instead of just one Dict[str, List[float]]
- However, we don't need to sample from the crels, we will just use the already generated parses, after de-duping
- ParserInputs needs modifying so that it takes a list of crel2probs instead of on dict for all parses
- Need to figure out what the optimal parse is based on amount of overlap with the actual crels, minus the false positives

# TODO 
- include the cum prob from the parse action result as a feature? - or simply compute the geometric mean of the probs?
- To speed up MIRA, de-dupe the generated parses prior to feature extraction. Where there are dupes, take the one with the highest cum prob
